##Loading dataset

In [ ]:
pip install folium==0.2.1

In [ ]:
pip install datasets

In [4]:
from datasets import load_dataset

In [5]:
raw_datasets = load_dataset("GonzaloA/fake_news")

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Using custom data configuration GonzaloA--fake_news-ec767410c901c71e


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/GonzaloA--fake_news-ec767410c901c71e/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 24353
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label'],
        num_rows: 8117
    })
})

In [7]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'Unnamed: 0': 0,
 'label': 0,
 'text': 'Maury is perhaps one of the trashiest shows on television today. It s right in line with the likes of the gutter trash that is Jerry Springer, and the fact that those shows are still on the air with the shit they air really is a sad testament to what Americans find to be entertaining. However, Maury really crossed the line with a Facebook post regarding one of their guest s appearance with a vile, disgusting caption on Tuesday evening.There was a young woman on there doing one of their episodes regarding the paternity of her child. However, on the page, the show posted an image of the woman, who happens to bear a striking resemblance to Senator and presidential candidate Ted Cruz. The caption from the Maury Show page read: The Lie Detector Test determined .that was a LIE!  Ted Cruz is just NOT that SEXY! As if that weren t horrible enough, the caption underneath the Imgur upload reads,  Ted Cruz in drag on Maury. Here is an image from the offici

#Preprocessing

In [ ]:
pip install transformers

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ghanashyamvtatti/roberta-fake-news")

model = AutoModelForSequenceClassification.from_pretrained("ghanashyamvtatti/roberta-fake-news")

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [27]:
def preprocess_function(examples):
  return tokenizer(examples['title'])

In [28]:
batch_size = 12

In [29]:
encoded_dataset = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [30]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 24353
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8117
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8117
    })
})

#Fine tune

In [31]:
from transformers import TrainingArguments, Trainer

In [32]:
num_labels = 2

In [33]:
training_args = TrainingArguments("test-trainer", num_train_epochs=1, max_steps=100)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer
)

max_steps is given, it will override any value given in num_train_epochs


In [35]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: title, Unnamed: 0, text. If title, Unnamed: 0, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 24353
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.16580366134643554, metrics={'train_runtime': 21.056, 'train_samples_per_second': 37.994, 'train_steps_per_second': 4.749, 'total_flos': 14138108115360.0, 'train_loss': 0.16580366134643554, 'epoch': 0.03})

#Evaluation

In [36]:
import numpy as np
from datasets import load_metric

metric = load_metric("acmcuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [38]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [39]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: title, Unnamed: 0, text. If title, Unnamed: 0, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8117
  Batch size = 8


{'eval_accuracy': 0.9757299494887274,
 'eval_loss': 0.13111388683319092,
 'eval_runtime': 55.1113,
 'eval_samples_per_second': 147.284,
 'eval_steps_per_second': 18.417}

In [45]:
eval_pred = trainer.predict(test_dataset=encoded_dataset["validation"])

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: title, Unnamed: 0, text. If title, Unnamed: 0, text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 8117
  Batch size = 8


In [54]:
eval_pred

PredictionOutput(predictions=array([[-3.6344917,  4.2066994],
       [-3.349828 ,  3.8920755],
       [-3.6674345,  4.24716  ],
       ...,
       [ 4.369863 , -5.1653795],
       [ 4.5673275, -5.4478917],
       [-3.5519428,  4.054652 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 1]), metrics={'test_loss': 0.13111388683319092, 'test_accuracy': 0.9757299494887274, 'test_runtime': 58.2457, 'test_samples_per_second': 139.358, 'test_steps_per_second': 17.426})

In [74]:
pred_data=[]
for i in eval_pred.predictions:
  pred_data.append(i)

In [76]:
predictions = np.argmax(pred_data, axis=-1)

In [61]:
eval_data = encoded_dataset["validation"]["label"]

In [80]:
from sklearn import metrics

In [82]:
predictions = predictions.tolist()

In [85]:
y_pred = predictions
y_act = eval_data
print(metrics.confusion_matrix(y_act, y_pred, labels=[1,0]))

[[4272  122]
 [  75 3648]]


In [86]:
print(metrics.classification_report(y_act, y_pred, labels=[1,0]))

              precision    recall  f1-score   support

           1       0.98      0.97      0.98      4394
           0       0.97      0.98      0.97      3723

    accuracy                           0.98      8117
   macro avg       0.98      0.98      0.98      8117
weighted avg       0.98      0.98      0.98      8117

